In [ ]:
import traceback
import threading
import cv2
import os
import numpy as np
from PIL import Image

In [ ]:
img = []
faces = None
gray = None
def face_recognition(id_name_action_list, confidence_threshold=80):
    global img
    global faces
    global gray

    # Display the video frame full screen
    # cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
    # cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)

    # path = os.path.dirname(os.path.abspath(__file__)) 
    path = os.path.dirname(os.path.abspath(os.getcwd()))
    
    class FaceReconitionThread(threading.Thread):
        def __init__(self, threadID, name):
            threading.Thread.__init__(self)
            self.threadID = threadID
            self.name = name

        def run(self):
            print('Starting face recognition thread')

            try:
                global img
                global faces
                global gray
                while True:
                    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    # Get all face from the video frame
                    faces = face_cascade.detectMultiScale(gray, 1.2,5)
                    
            except:
                    print(traceback.format_exc())
                

    # Create Local Binary Patterns Histograms for face recognization
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Load the trained mode (takes some time)
    recognizer.read(path+'/trainer/trainer.yml')

    # Create classifier from prebuilt model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    # Set the font style
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Initialize and start the video frame capture
    cap = cv2.VideoCapture(0)

    #init face recognition thread
    detectedFaces = []
    faces = []
    ids =[]
    prev_ids_1 = []
    prev_ids_2 = []

    gray = []
    ret, img =cap.read()

    faceRecognitionThread = FaceReconitionThread(1, "Face Reconition Thread")
    faceRecognitionThread.daemon = True
    faceRecognitionThread.start()

    # Loop
    while True:
        try:
            # Read the video frame
            ret, img =cap.read()

            prev_ids_2 = prev_ids_1
            prev_ids_1 = ids

            ids = []

            # For each face in faces
            for(x,y,w,h) in faces:

                # Create rectangle around the face
                cv2.rectangle(img, (x-20,y-20), (x+w+20,y+h+20), (0,255,0), 4)

                # Recognize the face belongs to which ID
                Id , confidence = recognizer.predict(gray[y:y+h,x:x+w])

                print("confidence:", confidence) 
                if confidence > confidence_threshold:
                    continue

                ids.append(Id)

                if Id  not in  prev_ids_1:
                    continue

                # Check the ID if exist 
                name = ""
                for target_ID, target_name, target_action in id_name_action_list:
                    if Id == target_ID:
                        name = target_name
                        if Id not in prev_ids_2:
                            target_action()
            

                if name == "":
                    name = "Unknown ID: " + str(Id)

                # Put text describe who is in the picture
                cv2.rectangle(img, (x-22,y-90), (x+w+22, y-22), (0,255,0), -1)
                cv2.putText(img, str(name), (x,y-40), font, 2, (255,255,255), 3)

            cv2.imshow('window',img)


            prev_faces = faces

            # If 'q' is pressed, close program
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        except:
            print( traceback.format_exc())
            break
    # Stop video
    cap.release()

    # Close all started windows
    cv2.destroyAllWindows()

In [ ]:
def action1():
    pass

id_name_action_list = [
  (1, "Raymond", action1),
]

face_recognition(id_name_action_list, confidence_threshold=80)